In [1]:
from scipy.fft import fft
import scipy
import os
import numpy as np
from numpy import set_printoptions
from matplotlib import pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.decomposition import PCA
from scipy.stats import mannwhitneyu
import h5py
import math
import pandas as pd
import random
import json
from pathlib import Path
from scipy.signal import butter, sosfilt, sosfreqz

from google.colab import drive
drive.mount('/content/drive')
!pip install mat73
DATASET_DIR = '/content/drive/MyDrive/IMUyAccelerometer/Data'
import mat73
from scipy.signal import butter,filtfilt

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#CODE UTILITIES

#return column of non-pandas matrix
def column(matrix, i):
    return [row[i] for row in matrix]

#Set up Nyquist Frequency and Butterworth lowpass filter to remove noise
fs = 100.0 
lowcut = 1
highcut = 45
# change to bandpass filter - come up with a range of values 
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
        nyq = 0.5 * fs
        low = lowcut / nyq
        high = highcut / nyq
        sos = butter(order, [low, high], analog=False, btype='band', output='sos')
        return sosfilt(sos, data)

def wavelet_denoise(data, noise_sigma, wavelet='haar'):
    '''Filter accelerometer data using wavelet denoising
    Modification of F. Blanco-Silva's code at: https://goo.gl/gOQwy5
    '''
    import pywt
    wavelet = pywt.Wavelet(wavelet)
    levels  = min(15, (numpy.floor(numpy.log2(data.shape[0]))).astype(int))
    # Francisco's code used wavedec2 for image data
    wavelet_coeffs = pywt.wavedec(data, wavelet, level=levels)
    threshold = noise_sigma*numpy.sqrt(2*numpy.log2(data.size))
    new_wavelet_coeffs = map(lambda x: pywt.threshold(x, threshold, mode='soft'),
                             wavelet_coeffs)
    return pywt.waverec(list(new_wavelet_coeffs), wavelet)

#define a few different feature selection methods to test
def featureSelection(features, k, labels, method):
    X = features.values
    Y = labels
    if method == 0:
      #RFE
      model = LogisticRegression(solver='lbfgs')
      rfe = RFE(model, n_features_to_select=k)
      fit = rfe.fit(X, Y)
      return fit
    
    if method == 1:
      #PCA
      pca = PCA(n_components=k)
      fit = pca.fit(X)
      return fit

    if method == 2:
      #Univariate
      test = SelectKBest(score_func=f_classif, k=k)
      fit = test.fit(X, Y)
      set_printoptions(precision=3)
      return fit.transform(X)

In [ ]:
#load all LLA data
s01 = mat73.loadmat(DATASET_DIR+'/S01.mat')
s02 = mat73.loadmat(DATASET_DIR+'/S02.mat')
s03 = mat73.loadmat(DATASET_DIR+'/S03.mat')
s04 = mat73.loadmat(DATASET_DIR+'/S04.mat')
s05 = mat73.loadmat(DATASET_DIR+'/S05.mat')
s06 = mat73.loadmat(DATASET_DIR+'/S06.mat')
s07 = mat73.loadmat(DATASET_DIR+'/S07.mat')
s08 = mat73.loadmat(DATASET_DIR+'/S08.mat')
s09 = mat73.loadmat(DATASET_DIR+'/S09.mat')
s10 = mat73.loadmat(DATASET_DIR+'/S10.mat')
s11 = mat73.loadmat(DATASET_DIR+'/S11.mat')
s12 = mat73.loadmat(DATASET_DIR+'/S12.mat')
s13 = mat73.loadmat(DATASET_DIR+'/S13.mat')
s14 = mat73.loadmat(DATASET_DIR+'/S14.mat')
s15 = mat73.loadmat(DATASET_DIR+'/S15.mat')
s16 = mat73.loadmat(DATASET_DIR+'/S16.mat')
s17 = mat73.loadmat(DATASET_DIR+'/S17.mat')

In [ ]:
#Load all control data
c01 = mat73.loadmat(DATASET_DIR+'/C01.mat')
c02 = mat73.loadmat(DATASET_DIR+'/C02.mat')
c03 = mat73.loadmat(DATASET_DIR+'/C03.mat')
c04 = mat73.loadmat(DATASET_DIR+'/C04.mat')
c05 = mat73.loadmat(DATASET_DIR+'/C05.mat')
c06 = mat73.loadmat(DATASET_DIR+'/C06.mat')
c07 = mat73.loadmat(DATASET_DIR+'/C07.mat')
c08 = mat73.loadmat(DATASET_DIR+'/C08.mat')
c09 = mat73.loadmat(DATASET_DIR+'/C09.mat')
c10 = mat73.loadmat(DATASET_DIR+'/C10.mat')
c11 = mat73.loadmat(DATASET_DIR+'/C11.mat')
c12 = mat73.loadmat(DATASET_DIR+'/C12.mat')
c13 = mat73.loadmat(DATASET_DIR+'/C13.mat')
c14 = mat73.loadmat(DATASET_DIR+'/C14.mat')

In [ ]:
#Isolate the sections of the data we need - See the Instructions.pdf from the original dataset organization
#The variable K exists in case of low-RAM runs. The full dataset requires ~64 GB of RAM and >4 hours to perform calculations
# to enable the full run, replace k with len(x[i])
x = [s01['SubjectData']['IMU']['Bouts']['Data'], s02['SubjectData']['IMU']['Bouts']['Data'], s03['SubjectData']['IMU']['Bouts']['Data'], s04['SubjectData']['IMU']['Bouts']['Data'], s05['SubjectData']['IMU']['Bouts']['Data'], s06['SubjectData']['IMU']['Bouts']['Data'], s07['SubjectData']['IMU']['Bouts']['Data'], s08['SubjectData']['IMU']['Bouts']['Data'], s09['SubjectData']['IMU']['Bouts']['Data'], s10['SubjectData']['IMU']['Bouts']['Data'], s11['SubjectData']['IMU']['Bouts']['Data'], s12['SubjectData']['IMU']['Bouts']['Data'], s13['SubjectData']['IMU']['Bouts']['Data'], s14['SubjectData']['IMU']['Bouts']['Data'], s15['SubjectData']['IMU']['Bouts']['Data'], s16['SubjectData']['IMU']['Bouts']['Data'], s17['SubjectData']['IMU']['Bouts']['Data'] ]
_id = ['M','F','M','M','M','M','M','M','M','M','M','M','M','M','M','M','F']
y = [s01['SubjectData']['IMU']['Bouts']['Timestamps'], s02['SubjectData']['IMU']['Bouts']['Timestamps'], s03['SubjectData']['IMU']['Bouts']['Timestamps'], s04['SubjectData']['IMU']['Bouts']['Timestamps'], s05['SubjectData']['IMU']['Bouts']['Timestamps'], s06['SubjectData']['IMU']['Bouts']['Timestamps'], s07['SubjectData']['IMU']['Bouts']['Timestamps'], s08['SubjectData']['IMU']['Bouts']['Timestamps'], s09['SubjectData']['IMU']['Bouts']['Timestamps'], s10['SubjectData']['IMU']['Bouts']['Timestamps'], s11['SubjectData']['IMU']['Bouts']['Timestamps'], s12['SubjectData']['IMU']['Bouts']['Timestamps'], s13['SubjectData']['IMU']['Bouts']['Timestamps'], s14['SubjectData']['IMU']['Bouts']['Timestamps'], s15['SubjectData']['IMU']['Bouts']['Timestamps'], s16['SubjectData']['IMU']['Bouts']['Timestamps'], s17['SubjectData']['IMU']['Bouts']['Timestamps'] ]
print(s01['SubjectData']['IMU']['Bouts']['DataHeaders'])
headers = s01['SubjectData']['IMU']['Bouts']['DataHeaders']
data_lla = []
time_lla = []
sex_lla = []
k=30
for i in range(len(x)): 
  for j in range(k):
    data_lla.append((x[i][j][0]))
    time_lla.append(y[i][j][0])
    sex_lla.append(_id[i])

print(len(data_lla))
print(len(sex_lla))

['Accelerometer X', 'Accelerometer Y', 'Accelerometer Z', 'Temperature', 'Gyroscope X', 'Gyroscope Y', 'Gyroscope Z', 'Magnetometer X', 'Magnetometer Y', 'Magnetometer Z']
510
510


In [ ]:
x = [c01['SubjectData']['IMU']['Bouts']['Data'], c02['SubjectData']['IMU']['Bouts']['Data'], c03['SubjectData']['IMU']['Bouts']['Data'], c04['SubjectData']['IMU']['Bouts']['Data'], c05['SubjectData']['IMU']['Bouts']['Data'], c06['SubjectData']['IMU']['Bouts']['Data'], c07['SubjectData']['IMU']['Bouts']['Data'], c08['SubjectData']['IMU']['Bouts']['Data'], c09['SubjectData']['IMU']['Bouts']['Data'], c10['SubjectData']['IMU']['Bouts']['Data'], c11['SubjectData']['IMU']['Bouts']['Data'], c12['SubjectData']['IMU']['Bouts']['Data'], c13['SubjectData']['IMU']['Bouts']['Data'], c14['SubjectData']['IMU']['Bouts']['Data'] ]
y = [c01['SubjectData']['IMU']['Bouts']['Timestamps'], c02['SubjectData']['IMU']['Bouts']['Timestamps'], c03['SubjectData']['IMU']['Bouts']['Timestamps'], c04['SubjectData']['IMU']['Bouts']['Timestamps'], c05['SubjectData']['IMU']['Bouts']['Timestamps'], c06['SubjectData']['IMU']['Bouts']['Timestamps'], c07['SubjectData']['IMU']['Bouts']['Timestamps'], c08['SubjectData']['IMU']['Bouts']['Timestamps'], c09['SubjectData']['IMU']['Bouts']['Timestamps'], c10['SubjectData']['IMU']['Bouts']['Timestamps'], c11['SubjectData']['IMU']['Bouts']['Timestamps'], c12['SubjectData']['IMU']['Bouts']['Timestamps'], c13['SubjectData']['IMU']['Bouts']['Timestamps'], c14['SubjectData']['IMU']['Bouts']['Timestamps'] ]
print(c01['SubjectData']['IMU']['Bouts']['DataHeaders'])
headers = c01['SubjectData']['IMU']['Bouts']['DataHeaders']
_id = ['M','M','F','M','M','M','M','M','M','M','M','M','M','M']
data_c = []
time_c = []
sex_c = []
k=37
for i in range(len(x)): 
  for j in range(k):
    data_c.append((x[i][j][0]))
    time_c.append(y[i][j][0])
    sex_c.append(_id[i])
print(len(data_c))
print(len(sex_c))

['Accelerometer X', 'Accelerometer Y', 'Accelerometer Z', 'Temperature', 'Gyroscope X', 'Gyroscope Y', 'Gyroscope Z', 'Magnetometer X', 'Magnetometer Y', 'Magnetometer Z']
518
518


In [ ]:
#clean up our data, remove incomplete data sets and drop unnessecary columns
# then, apply the butterworth filter to Accelerometer data
#We also send our cleaned up data to a pandas DataFrame for easier manipulation
sexL = []
datax = []
j = -1
for i in range(len(data_lla)):
  #cull all incomplete data / non waveform data
  if type(data_lla[i][0]) is np.float64 or len(data_lla[i][0])<10 or \
   sum(data_lla[i][0])==0:
    continue
  datax.append(pd.DataFrame(data_lla[i]))
  datax[j].columns = headers
  datax[j]['Time'] = time_lla[i]
  sexL.append(sex_lla[i])
print(len(datax))
print(len(sexL))
# butterworth filter on all Accelerometer data, drop unneccesary temp data
for k in range(len(datax)):
  x = butter_bandpass_filter(datax[k].iloc[:,0], lowcut, highcut, fs, order=5)
  datax[k].iloc[:,0] = x
  y = butter_bandpass_filter(datax[k].iloc[:,1], lowcut, highcut, fs, order=5)
  datax[k].iloc[:,0] = y
  z = butter_bandpass_filter(datax[k].iloc[:,2], lowcut, highcut, fs, order=5)
  datax[k].iloc[:,0] = z
  datax[k].drop(datax[k].columns[[3]], axis=1, inplace=True)

sexC = []
datac = []
j = -1
for i in range(len(data_c)):
  #cull all incomplete data / non waveform data
  if type(data_c[i][0]) is np.float64 or len(data_c[i][0])<10 or \
   sum(data_c[i][0])==0:
    continue
  datac.append(pd.DataFrame(data_c[i]))
  datac[j].columns = headers
  datac[j]['Time'] = time_c[i]
  sexC.append(sex_c[i])
print(len(datac))
# butterworth filter on all Accelerometer data, drop unneccesary temp data
for k in range(len(datac)):
  x = butter_bandpass_filter(datac[k].iloc[:,0], lowcut, highcut, fs, order=5)
  datac[k].iloc[:,0] = x
  y = butter_bandpass_filter(datac[k].iloc[:,1], lowcut, highcut, fs, order=5)
  datac[k].iloc[:,0] = y
  z = butter_bandpass_filter(datac[k].iloc[:,2], lowcut, highcut, fs, order=5)
  datac[k].iloc[:,0] = z
  datac[k].drop(datac[k].columns[[3]], axis=1, inplace=True)


497
497
514


In [ ]:
#calculate jerk from acceleration data
# Jerk - the third derivative of position, or first derivative of Acceleration, describes 
# change in Acceleration over time. This part of the process takes the longest of all calculations.
for k in range(len(datax)):
  jerkx = []
  jerky = []
  jerkz = []
  jerkx.append(0)
  jerky.append(0)
  jerkz.append(0)
  for i in range(len(datax[k].iloc[:,1])-1):
    jerkx.append((datax[k].iloc[:,0][i]-datax[k].iloc[:,0][i+1])/(datax[k].loc[:,'Time'][i]-datax[k].loc[:,'Time'][i+1]))
    jerky.append((datax[k].iloc[:,1][i]-datax[k].iloc[:,1][i+1])/(datax[k].loc[:,'Time'][i]-datax[k].loc[:,'Time'][i+1]))
    jerkz.append((datax[k].iloc[:,2][i]-datax[k].iloc[:,2][i+1])/(datax[k].loc[:,'Time'][i]-datax[k].loc[:,'Time'][i+1]))
  datax[k]['Jerk X'] = jerkx
  datax[k]['Jerk Y'] = jerky
  datax[k]['Jerk Z'] = jerkz 

<ipython-input-23-994f95e974e2>:12: RuntimeWarning: divide by zero encountered in double_scalars
  jerkx.append((datax[k].iloc[:,0][i]-datax[k].iloc[:,0][i+1])/(datax[k].loc[:,'Time'][i]-datax[k].loc[:,'Time'][i+1]))
<ipython-input-23-994f95e974e2>:13: RuntimeWarning: divide by zero encountered in double_scalars
  jerky.append((datax[k].iloc[:,1][i]-datax[k].iloc[:,1][i+1])/(datax[k].loc[:,'Time'][i]-datax[k].loc[:,'Time'][i+1]))
<ipython-input-23-994f95e974e2>:14: RuntimeWarning: divide by zero encountered in double_scalars
  jerkz.append((datax[k].iloc[:,2][i]-datax[k].iloc[:,2][i+1])/(datax[k].loc[:,'Time'][i]-datax[k].loc[:,'Time'][i+1]))
<ipython-input-23-994f95e974e2>:14: RuntimeWarning: invalid value encountered in double_scalars
  jerkz.append((datax[k].iloc[:,2][i]-datax[k].iloc[:,2][i+1])/(datax[k].loc[:,'Time'][i]-datax[k].loc[:,'Time'][i+1]))
<ipython-input-23-994f95e974e2>:13: RuntimeWarning: invalid value encountered in double_scalars
  jerky.append((datax[k].iloc[:,1][i]

In [ ]:
#calculate jerk on control dataset
for k in range(len(datac)):
  jerkx = []
  jerky = []
  jerkz = []
  jerkx.append(0)
  jerky.append(0)
  jerkz.append(0)
  for i in range(len(datac[k].iloc[:,1])-1):
    jerkx.append((datac[k].iloc[:,0][i]-datac[k].iloc[:,0][i+1])/(datac[k].loc[:,'Time'][i]-datac[k].loc[:,'Time'][i+1]))
    jerky.append((datac[k].iloc[:,1][i]-datac[k].iloc[:,1][i+1])/(datac[k].loc[:,'Time'][i]-datac[k].loc[:,'Time'][i+1]))
    jerkz.append((datac[k].iloc[:,2][i]-datac[k].iloc[:,2][i+1])/(datac[k].loc[:,'Time'][i]-datac[k].loc[:,'Time'][i+1]))
  datac[k]['Jerk X'] = jerkx
  datac[k]['Jerk Y'] = jerky
  datac[k]['Jerk Z'] = jerkz 
print(datac[0])

       Accelerometer X  Accelerometer Y  Accelerometer Z  Gyroscope X  \
0             0.543552        -0.011719         1.004883     0.122070   
1             0.982920        -0.012207         0.998535     0.122070   
2             0.510843        -0.011230         1.006836     0.183105   
3             0.447568        -0.011719         1.002930     0.183105   
4             0.353966        -0.012207         1.004883     0.183105   
...                ...              ...              ...          ...   
17996         0.006915         0.300293         0.895996    -0.061035   
17997        -0.010702         0.327148         0.867676     1.220703   
17998        -0.002651         0.294922         0.890625    -0.854492   
17999         0.000614         0.308594         0.879395     0.488281   
18000         0.000468         0.304199         0.887695     0.000000   

       Gyroscope Y  Gyroscope Z  Magnetometer X  Magnetometer Y  \
0         1.464844     0.061035       11.865234        7

In [ ]:
# Calculate Accelerometer and Jerk Magnitudes on Full Dataset
# Magnitude of a vector = sqrt(x^2+y^2+z^2)
#This is likely the 2nd longest operation in the whole thing
for k in range(len(datac)):
  accelmag = []
  magmag = []
  jerkmag = []
  gyromag = []
  for i in range(len(datac[k].iloc[:,1])):
    accelmag.append( math.sqrt(datac[k].iloc[:,0][i]**2 + datac[k].iloc[:,1][i]**2 + datac[k].iloc[:,2][i]**2) )
    gyromag.append( math.sqrt(datac[k].iloc[:,3][i]**2 + datac[k].iloc[:,4][i]**2 + datac[k].iloc[:,5][i]**2) )
    magmag.append( math.sqrt(datac[k].iloc[:,6][i]**2 + datac[k].iloc[:,7][i]**2 + datac[k].iloc[:,8][i]**2) )
    jerkmag.append( math.sqrt(datac[k].iloc[:,10][i]**2 + datac[k].iloc[:,11][i]**2 + datac[k].iloc[:,12][i]**2) )
  datac[k]['Accel Mag'] = accelmag
  datac[k]['Jerk Mag'] = jerkmag
  datac[k]['Mag Mag'] = magmag
  datac[k]['Gyro Mag'] = gyromag

In [ ]:
for k in range(len(datax)):
  accelmag = []
  jerkmag = []
  magmag = []
  gyromag = []
  for i in range(len(datax[k].iloc[:,1])):
    accelmag.append( math.sqrt(datax[k].iloc[:,0][i]**2 + datax[k].iloc[:,1][i]**2 + datax[k].iloc[:,2][i]**2) )
    gyromag.append( math.sqrt(datax[k].iloc[:,3][i]**2 + datax[k].iloc[:,4][i]**2 + datax[k].iloc[:,5][i]**2) )
    magmag.append( math.sqrt(datax[k].iloc[:,6][i]**2 + datax[k].iloc[:,7][i]**2 + datax[k].iloc[:,8][i]**2) )
    jerkmag.append( math.sqrt(datax[k].iloc[:,10][i]**2 + datax[k].iloc[:,11][i]**2 + datax[k].iloc[:,12][i]**2) )
  datax[k]['Accel Mag'] = accelmag
  datax[k]['Jerk Mag'] = jerkmag
  datax[k]['Mag Mag'] = magmag
  datax[k]['Gyro Mag'] = gyromag
print(len(datac))

514


In [ ]:
# OPTIONALLY, SAVE DATA TO CSVs TO RE-IMPORT FOR LATER EVALUATION
print(len(datax))
for k in range(len(datax)):
  filename = Path(DATASET_DIR+'/lla/'+str(k)+'.csv')
  datax[k].to_csv(filename)

497


In [ ]:
for k in range(len(datac)):
  filename = Path(DATASET_DIR+'/control/'+str(k)+'.csv')
  datac[k].to_csv(filename)

In [38]:
# OPTIONALLY, IMPORT DATA BACK FROM CSVs TO SAVE TIME/RAM on future runs
k = 0
datax = []
filenameL = Path(DATASET_DIR+'/lla/'+str(k)+'.csv')
for k in range(497):
  df = pd.read_csv(filenameL)
  datax.append(df)
print(len(datax))
k = 0
datac = []
filenameC = Path(DATASET_DIR+'/control/'+str(k)+'.csv')
for k in range(514):
  df = pd.read_csv(filenameC)
  datac.append(df)
print(len(datac))

497
514


In [40]:
#Calculate Sliding Windows. Divy up the data into sliding windows of size 100 (1s of data) + a 33 frame offset
print(len(datax))
temp = datax
datax = []
sexLla = []
for k in range(len(temp)):
  c = 1
  while (len(temp[k]) - c) > 100:
    datax.append(temp[k].iloc[c:c+100])
    #sexLla.append(sexL[k])
    c+=33
print(len(datax))
print(len(datac))
temp = datac
datac = []
sexControl = []
for k in range(len(temp)):
  c = 1
  while (len(temp[k]) - c)>=100:
    datac.append(temp[k].iloc[c:c+100])
    #sexControl.append(sexC[k])
    c+=33
print(len(datac))

497
179417
514
279102


In [ ]:
df = pd.DataFrame(sexLla)
filename = Path(DATASET_DIR+'/lla/sex.csv')
df.to_csv(filename)

df = pd.DataFrame(sexControl)
filename = Path(DATASET_DIR+'/control/sex.csv')
df.to_csv(filename)

In [ ]:
# get wavelet/fft features. These are complex numbers so we will need to use the magnitude 
# instead of the raw fft values in order to perform evaluations on them
import numpy
fftL = []
#fft on LLA dataset
for k in range(len(datax)):
  df = pd.DataFrame()
  df['accXfft'] = [abs(ele) for ele in numpy.fft.fft((datax[k].loc[:,'Accelerometer X']))]
  df['accYfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datax[k].loc[:,'Accelerometer Y']))]
  df['accZfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datax[k].loc[:,'Accelerometer Z']))]
  df['gyroXfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datax[k].loc[:,'Gyroscope X']))]
  df['gyroYfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datax[k].loc[:,'Gyroscope Y']))]
  df['gyroZfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datax[k].loc[:,'Gyroscope Z']))]
  df['magXfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datax[k].loc[:,'Magnetometer X']))]
  df['magYfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datax[k].loc[:,'Magnetometer Y']))]
  df['magZfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datax[k].loc[:,'Magnetometer Z']))]
  df['jerkXfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datax[k].loc[:,'Jerk X']))]
  df['jerkYfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datax[k].loc[:,'Jerk Y']))]
  df['jerkZfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datax[k].loc[:,'Jerk Z']))]
  df['magAccelfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datax[k].loc[:,'Accel Mag']))]
  df['magJerkfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datax[k].loc[:,'Jerk Mag']))]
  df['magMagfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datax[k].loc[:,'Mag Mag']))]
  df['magGyrofft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datax[k].loc[:,'Gyro Mag']))]
  fftL.append(df)
  if k%10000 == 0:
    print(k)

fftC = []
k=0
for k in range(len(datac)):
  df = pd.DataFrame()
  df['accXfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datac[k].loc[:,'Accelerometer X']))]
  df['accYfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datac[k].loc[:,'Accelerometer Y']))]
  df['accZfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datac[k].loc[:,'Accelerometer Z']))]
  df['gyroXfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datac[k].loc[:,'Gyroscope X']))]
  df['gyroYfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datac[k].loc[:,'Gyroscope Y']))]
  df['gyroZfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datac[k].loc[:,'Gyroscope Z']))]
  df['magXfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datac[k].loc[:,'Magnetometer X']))]
  df['magYfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datac[k].loc[:,'Magnetometer Y']))]
  df['magZfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datac[k].loc[:,'Magnetometer Z']))]
  df['jerkXfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datac[k].loc[:,'Jerk X']))]
  df['jerkYfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datac[k].loc[:,'Jerk Y']))]
  df['jerkZfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datac[k].loc[:,'Jerk Z']))]
  df['magAccelfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datac[k].loc[:,'Accel Mag']))]
  df['magJerkfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datac[k].loc[:,'Jerk Mag']))]
  df['magMagfft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datac[k].loc[:,'Mag Mag']))]
  df['magGyrofft'] = [abs(ele) for ele in numpy.fft.fft(np.array(datac[k].loc[:,'Gyro Mag']))]
  fftC.append(df)
  if k%10000 == 0:
    print(k)


0


KeyboardInterrupt: ignored

In [ ]:
#Get features from the time domain. This is also where we apply our labels to each waveform 
from scipy.stats import skew, kurtosis
def getfeatures(features, label, fftL, datax):
  for k in range(len(datax)):
    item = [label, min(datax[k].loc[:,'Accelerometer X']), max(datax[k].loc[:,'Accelerometer X']), np.sqrt(np.mean(datax[k].loc[:,'Accelerometer X']**2)), np.std(datax[k].loc[:,'Accelerometer X'], axis=0), skew(datax[k].loc[:,'Accelerometer X']), kurtosis(datax[k].loc[:,'Accelerometer X']),
          min(datax[k].loc[:,'Accelerometer Y']), max(datax[k].loc[:,'Accelerometer Y']), np.sqrt(np.mean(datax[k].loc[:,'Accelerometer Y']**2)), np.std(datax[k].loc[:,'Accelerometer Y'], axis=0),  skew(datax[k].loc[:,'Accelerometer Y']), kurtosis(datax[k].loc[:,'Accelerometer Y']),
          min(datax[k].loc[:,'Accelerometer Z']), max(datax[k].loc[:,'Accelerometer Z']), np.sqrt(np.mean(datax[k].loc[:,'Accelerometer Z']**2)), np.std(datax[k].loc[:,'Accelerometer Z'], axis=0), skew(datax[k].loc[:,'Accelerometer Z']), kurtosis(datax[k].loc[:,'Accelerometer Z']),
          min(datax[k].loc[:,'Gyroscope X']), max(datax[k].loc[:,'Gyroscope X']), np.sqrt(np.mean(datax[k].loc[:,'Gyroscope X']**2)), np.std(datax[k].loc[:,'Gyroscope X'], axis=0), skew(datax[k].loc[:,'Gyroscope X']), kurtosis(datax[k].loc[:,'Gyroscope X']),
          min(datax[k].loc[:,'Gyroscope Y']), max(datax[k].loc[:,'Gyroscope Y']), np.sqrt(np.mean(datax[k].loc[:,'Gyroscope Y']**2)), np.std(datax[k].loc[:,'Gyroscope Y'], axis=0), skew(datax[k].loc[:,'Gyroscope Y']), kurtosis(datax[k].loc[:,'Gyroscope Y']),
          min(datax[k].loc[:,'Gyroscope Z']), max(datax[k].loc[:,'Gyroscope Z']), np.sqrt(np.mean(datax[k].loc[:,'Gyroscope Z']**2)), np.std(datax[k].loc[:,'Gyroscope Z'], axis=0), skew(datax[k].loc[:,'Gyroscope Z']), kurtosis(datax[k].loc[:,'Gyroscope Z']),
          min(datax[k].loc[:,'Magnetometer X']), max(datax[k].loc[:,'Magnetometer X']), np.sqrt(np.mean(datax[k].loc[:,'Magnetometer X']**2)), np.std(datax[k].loc[:,'Magnetometer X'], axis=0), skew(datax[k].loc[:,'Magnetometer X']), kurtosis(datax[k].loc[:,'Magnetometer X']),
          min(datax[k].loc[:,'Magnetometer Y']), max(datax[k].loc[:,'Magnetometer Y']), np.sqrt(np.mean(datax[k].loc[:,'Magnetometer Y']**2)), np.std(datax[k].loc[:,'Magnetometer Y'], axis=0), skew(datax[k].loc[:,'Magnetometer Y']), kurtosis(datax[k].loc[:,'Magnetometer Y']),
          min(datax[k].loc[:,'Magnetometer Z']), max(datax[k].loc[:,'Magnetometer Z']), np.sqrt(np.mean(datax[k].loc[:,'Magnetometer Z']**2)), np.std(datax[k].loc[:,'Magnetometer Z'], axis=0), skew(datax[k].loc[:,'Magnetometer Z']), kurtosis(datax[k].loc[:,'Magnetometer Z']),
          min(datax[k].loc[:,'Jerk X']), max(datax[k].loc[:,'Jerk X']), np.sqrt(np.mean(datax[k].loc[:,'Jerk X']**2)), np.std(datax[k].loc[:,'Jerk X'], axis=0), skew(datax[k].loc[:,'Jerk X']), kurtosis(datax[k].loc[:,'Jerk X']),
          min(datax[k].loc[:,'Jerk Y']), max(datax[k].loc[:,'Jerk Y']), np.sqrt(np.mean(datax[k].loc[:,'Jerk Y']**2)), np.std(datax[k].loc[:,'Jerk Y'], axis=0), skew(datax[k].loc[:,'Jerk Y']), kurtosis(datax[k].loc[:,'Jerk X']),
          min(datax[k].loc[:,'Jerk Z']), max(datax[k].loc[:,'Jerk Z']), np.sqrt(np.mean(datax[k].loc[:,'Jerk Z']**2)), np.std(datax[k].loc[:,'Jerk Z'], axis=0), skew(datax[k].loc[:,'Jerk Z']), kurtosis(datax[k].loc[:,'Jerk X']),
          min(datax[k].loc[:,'Accel Mag']), max(datax[k].loc[:,'Accel Mag']), np.sqrt(np.mean(datax[k].loc[:,'Accel Mag']**2)), np.std(datax[k].loc[:,'Accel Mag'], axis=0), skew(datax[k].loc[:,'Accel Mag']), kurtosis(datax[k].loc[:,'Accel Mag']),
          min(datax[k].loc[:,'Jerk Mag']), max(datax[k].loc[:,'Jerk Mag']), np.sqrt(np.mean(datax[k].loc[:,'Jerk Mag']**2)), np.std(datax[k].loc[:,'Jerk Mag'], axis=0), skew(datax[k].loc[:,'Jerk Mag']), kurtosis(datax[k].loc[:,'Jerk Mag']),
          min(datax[k].loc[:,'Mag Mag']), max(datax[k].loc[:,'Mag Mag']), np.sqrt(np.mean(datax[k].loc[:,'Mag Mag']**2)), np.std(datax[k].loc[:,'Mag Mag'], axis=0), skew(datax[k].loc[:,'Mag Mag']), kurtosis(datax[k].loc[:,'Mag Mag']),
          min(datax[k].loc[:,'Gyro Mag']), max(datax[k].loc[:,'Gyro Mag']), np.sqrt(np.mean(datax[k].loc[:,'Gyro Mag']**2)), np.std(datax[k].loc[:,'Gyro Mag'], axis=0), skew(datax[k].loc[:,'Gyro Mag']), kurtosis(datax[k].loc[:,'Gyro Mag']),
          min(fftL[k].loc[:,'accXfft']), max(fftL[k].loc[:,'accXfft']), np.sqrt(np.mean(fftL[k].loc[:,'accXfft']**2)), np.std(fftL[k].loc[:,'accXfft'], axis=0), skew(fftL[k].loc[:,'accXfft']), kurtosis(fftL[k].loc[:,'accXfft']),
          min(fftL[k].loc[:,'accYfft']), max(fftL[k].loc[:,'accYfft']), np.sqrt(np.mean(fftL[k].loc[:,'accYfft']**2)), np.std(fftL[k].loc[:,'accYfft'], axis=0), skew(fftL[k].loc[:,'accYfft']), kurtosis(fftL[k].loc[:,'accYfft']),
          min(fftL[k].loc[:,'accZfft']), max(fftL[k].loc[:,'accZfft']), np.sqrt(np.mean(fftL[k].loc[:,'accZfft']**2)), np.std(fftL[k].loc[:,'accZfft'], axis=0), skew(fftL[k].loc[:,'accZfft']), kurtosis(fftL[k].loc[:,'accZfft']),
          min(fftL[k].loc[:,'gyroXfft']), max(fftL[k].loc[:,'gyroXfft']), np.sqrt(np.mean(fftL[k].loc[:,'gyroXfft']**2)), np.std(fftL[k].loc[:,'gyroXfft'], axis=0), skew(fftL[k].loc[:,'gyroXfft']), kurtosis(fftL[k].loc[:,'gyroXfft']),
          min(fftL[k].loc[:,'gyroYfft']), max(fftL[k].loc[:,'gyroYfft']), np.sqrt(np.mean(fftL[k].loc[:,'gyroYfft']**2)), np.std(fftL[k].loc[:,'gyroYfft'], axis=0), skew(fftL[k].loc[:,'gyroYfft']), kurtosis(fftL[k].loc[:,'gyroYfft']),
          min(fftL[k].loc[:,'gyroZfft']), max(fftL[k].loc[:,'gyroZfft']), np.sqrt(np.mean(fftL[k].loc[:,'gyroZfft']**2)), np.std(fftL[k].loc[:,'gyroZfft'], axis=0), skew(fftL[k].loc[:,'gyroZfft']), kurtosis(fftL[k].loc[:,'gyroZfft']),
          min(fftL[k].loc[:,'magXfft']), max(fftL[k].loc[:,'magXfft']), np.sqrt(np.mean(fftL[k].loc[:,'magXfft']**2)), np.std(fftL[k].loc[:,'magXfft'], axis=0), skew(fftL[k].loc[:,'magXfft']), kurtosis(fftL[k].loc[:,'magXfft']),
          min(fftL[k].loc[:,'magYfft']), max(fftL[k].loc[:,'magYfft']), np.sqrt(np.mean(fftL[k].loc[:,'magYfft']**2)), np.std(fftL[k].loc[:,'magYfft'], axis=0), skew(fftL[k].loc[:,'magYfft']), kurtosis(fftL[k].loc[:,'magYfft']),
          min(fftL[k].loc[:,'magZfft']), max(fftL[k].loc[:,'magZfft']), np.sqrt(np.mean(fftL[k].loc[:,'magZfft']**2)), np.std(fftL[k].loc[:,'magZfft'], axis=0), skew(fftL[k].loc[:,'magZfft']), kurtosis(fftL[k].loc[:,'magZfft']),
          min(fftL[k].loc[:,'jerkXfft']), max(fftL[k].loc[:,'jerkXfft']), np.sqrt(np.mean(fftL[k].loc[:,'jerkXfft']**2)), np.std(fftL[k].loc[:,'jerkXfft'], axis=0), skew(fftL[k].loc[:,'jerkXfft']), kurtosis(fftL[k].loc[:,'jerkXfft']),
          min(fftL[k].loc[:,'jerkYfft']), max(fftL[k].loc[:,'jerkYfft']), np.sqrt(np.mean(fftL[k].loc[:,'jerkYfft']**2)), np.std(fftL[k].loc[:,'jerkYfft'], axis=0), skew(fftL[k].loc[:,'jerkYfft']), kurtosis(fftL[k].loc[:,'jerkYfft']),
          min(fftL[k].loc[:,'jerkZfft']), max(fftL[k].loc[:,'jerkZfft']), np.sqrt(np.mean(fftL[k].loc[:,'jerkZfft']**2)), np.std(fftL[k].loc[:,'jerkZfft'], axis=0), skew(fftL[k].loc[:,'jerkZfft']), kurtosis(fftL[k].loc[:,'jerkZfft']),
          min(fftL[k].loc[:,'magAccelfft']), max(fftL[k].loc[:,'magAccelfft']), np.sqrt(np.mean(fftL[k].loc[:,'magAccelfft']**2)), np.std(fftL[k].loc[:,'magAccelfft'], axis=0), skew(fftL[k].loc[:,'magAccelfft']), kurtosis(fftL[k].loc[:,'magAccelfft']),
          min(fftL[k].loc[:,'magJerkfft']), max(fftL[k].loc[:,'magJerkfft']), np.sqrt(np.mean(fftL[k].loc[:,'magJerkfft']**2)), np.std(fftL[k].loc[:,'magJerkfft'], axis=0), skew(fftL[k].loc[:,'magJerkfft']), kurtosis(fftL[k].loc[:,'magJerkfft']),
          min(fftL[k].loc[:,'magMagfft']), max(fftL[k].loc[:,'magMagfft']), np.sqrt(np.mean(fftL[k].loc[:,'magMagfft']**2)), np.std(fftL[k].loc[:,'magMagfft'], axis=0), skew(fftL[k].loc[:,'magMagfft']), kurtosis(fftL[k].loc[:,'magMagfft']),
          min(fftL[k].loc[:,'magGyrofft']), max(fftL[k].loc[:,'magGyrofft']), np.sqrt(np.mean(fftL[k].loc[:,'magGyrofft']**2)), np.std(fftL[k].loc[:,'magGyrofft'], axis=0), skew(fftL[k].loc[:,'magGyrofft']), kurtosis(fftL[k].loc[:,'magGyrofft']),
          ]
    if k%10000 == 0:
      print(k)
    features.append(item)
  return features


In [ ]:
#Calculate and append fft features 
# increase features - remove mean, use rms, kurtosis, skewness, dominant frequency,
features = []
features = getfeatures(features, 0, fftL, datax)
features = getfeatures(features, 1, fftC, datac)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000


In [ ]:
df = pd.DataFrame(features)
filename = Path(DATASET_DIR+'/features.csv')
df.to_csv(filename)

In [16]:
filename = Path(DATASET_DIR+'/features.csv')
features = pd.read_csv(filename)

In [50]:
filename = Path(DATASET_DIR+'/control/sex.csv')
sexControl = pd.read_csv(filename)
filename = Path(DATASET_DIR+'/lla/sex.csv')
sexLla = pd.read_csv(filename)
sexLla.drop("Unnamed: 0", axis=1, inplace=True)
sexControl.drop("Unnamed: 0", axis=1, inplace=True)
print(len(sexLla))
print(len(sexControl))

179417
279102


In [54]:
# pov: u messed up the labels

#features.drop("Unnamed: 0", axis=1, inplace=True)
#features.drop("0", axis=1, inplace=True)
print(len(sexLla))
print(sexLla)
print(len(sexControl))
print(sexControl)
labels = []
for i in range(len(datax)):
  labels.append("LLA")
for i in range(len(datac)):
  labels.append("Control")
temp = [sexLla, sexControl]
sex_labels = pd.concat(temp)
print(len(sex_labels))
print(len(labels))

179417
        0
0       M
1       M
2       M
3       M
4       M
...    ..
179412  F
179413  F
179414  F
179415  F
179416  F

[179417 rows x 1 columns]
279102
        0
0       M
1       M
2       M
3       M
4       M
...    ..
279097  M
279098  M
279099  M
279100  M
279101  M

[279102 rows x 1 columns]
458519
458519


In [30]:
#Mann Whitney U test to compare sets 
from scipy.stats import ttest_ind
print(len(datax))
feature_names = ["label","Acc X min", "Acc X max", "Acc X RMS", "Acc X STD", "Acc X Skewness", "Acc X Kurtosis", "Acc Y min", "Acc Y max", "Acc Y RMS", "Acc Y STD", "Acc Y Skewness", "Acc Y Kurtosis", "Acc Z min", "Acc Z max", "Acc Z RMS", "Acc Z STD", "Acc Z Skewness", "Acc Z Kurtosis",
                 "Gyro X min", "Gyro X max", "Gyro X RMS", "Gyro X STD", "Gyro X Skewness", "Gyro X Kurtosis","Gyro Y min", "Gyro Y max", "Gyro Y RMS", "Gyro Y STD", "Gyro Y Skewness", "Gyro Y Kurtosis", "Gyro Z min", "Gyro Z max", "Gyro Z RMS", "Gyro Z STD", "Gyro Z Skewness", "Gyro Z Kurtosis",
                 "Mag X min", "Mag X max", "Mag X RMS", "Mag X STD", "Mag X Skewness", "Mag X Kurtosis", "Mag Y min", "Mag Y max", "Mag Y RMS", "Mag Y STD", "Mag Y Skewness", "Mag Y Kurtosis", "Mag Z min", "Mag Z max", "Mag Z RMS", "Mag Z STD", "Mag Z Skewness", "Mag Z Kurtosis",
                 "Jerk X min", "Jerk X max", "Jerk X RMS", "Jerk X STD", "Jerk X Skewness", "Jerk X Kurtosis", "Jerk Y min", "Jerk Y max", "Jerk Y RMS", "Jerk Y STD", "Jerk Y Skewness", "Jerk Y Kurtosis", "Jerk Z min", "Jerk Z max", "Jerk Z RMS", "Jerk Z STD", "Jerk Z Skewness", "Jerk Z Kurtosis",
                 "Accel Mag min", "Accel Mag max", "Accel Mag RMS", "Accel Mag STD", "Accel Mag Skewness", "Accel Mag Kurtosis", "Jerk Mag min", "Jerk Mag max", "Jerk Mag RMS", "Jerk Mag STD", "Jerk Mag Skewness", "Jerk Mag Kurtosis", 
                 "Mag Mag min", "Mag Mag max", "Mag Mag RMS", "Mag Mag STD", "Mag Mag Skewness", "Mag Mag Kurtosis", "Gyro Mag min", "Gyro Mag max", "Gyro Mag RMS", "Gyro Mag STD", "Gyro Mag Skewness", "Gyro Mag Kurtosis",
                 "FFT Acc X min", "FFT Acc X max", "FFT Acc X RMS", "FFT Acc X STD", "FFT Acc X Skewness", "FFT Acc X Kurtosis", "FFT Acc Y min", "FFT Acc Y max", "FFT Acc Y RMS", "FFT Acc Y STD", "FFT Acc Y Skewness", "FFT Acc Y Kurtosis", "FFT Acc Z min", "FFT Acc Z max", "FFT Acc Z RMS", "FFT Acc Z STD", "FFT Acc Z Skewness", "FFT Acc Z Kurtosis",
                 "FFT Gyro X min", "FFT Gyro X max", "FFT Gyro X RMS", "FFT Gyro X STD", "FFT Gyro X Skewness", "FFT Gyro X Kurtosis","FFT Gyro Y min", "FFT Gyro Y max", "FFT Gyro Y RMS", "FFT Gyro Y STD", "FFT Gyro Y Skewness", "FFT Gyro Y Kurtosis", "FFT Gyro Z min", "FFT Gyro Z max", "FFT Gyro Z RMS", "FFT Gyro Z STD", "FFT Gyro Z Skewness", "FFT Gyro Z Kurtosis",
                 "FFT Mag X min", "FFT Mag X max", "FFT Mag X RMS", "FFT Mag X STD", "FFT Mag X Skewness", "FFT Mag X Kurtosis", "FFT Mag Y min", "FFT Mag Y max", "FFT Mag Y RMS", "FFT Mag Y STD", "FFT Mag Y Skewness", "FFT Mag Y Kurtosis", "FFT Mag Z min", "FFT Mag Z max", "FFT Mag Z RMS", "FFT Mag Z STD", "FFT Mag Z Skewness", "FFT Mag Z Kurtosis",
                 "FFT Jerk X min", "FFT Jerk X max", "FFT Jerk X RMS", "FFT Jerk X STD", "FFT Jerk X Skewness", "FFT Jerk X Kurtosis", "FFT Jerk Y min", "FFT Jerk Y max", "FFT Jerk Y RMS", "FFT Jerk Y STD", "FFT Jerk Y Skewness", "FFT Jerk Y Kurtosis", "FFT Jerk Z min", "Jerk Z max", "FFT Jerk Z RMS", "FFT Jerk Z STD", "FFT Jerk Z Skewness", "FFT Jerk Z Kurtosis",
                 "FFT Accel Mag min", "FFT Accel Mag max", "FFT Accel Mag RMS", "FFT Accel Mag STD", "FFT Accel Mag Skewness", "FFT Accel Mag Kurtosis", "FFT Jerk Mag min", "FFT Jerk Mag max", "FFT Jerk Mag RMS", "FFT Jerk Mag STD", "FFT Jerk Mag Skewness", "FFT Jerk Mag Kurtosis", 
                 "FFT Mag Mag min", "FFT Mag Mag max", "FFT Mag Mag RMS", "FFT Mag Mag STD", "FFT Mag Mag Skewness", "FFT Mag Mag Kurtosis", "FFT Gyro Mag min", "FFT Gyro Mag max", "FFT Gyro Mag RMS", "FFT Gyro Mag STD", "FFT Gyro Mag Skewness", "FFT Gyro Mag Kurtosis"
                 ]
#features.drop("Unnamed: 0", axis=1, inplace=True)
print(ttest_ind(datax[0]['Accelerometer X'], datac[0]['Accelerometer X'], alternative="less"))
print(ttest_ind(datax[0]['Accelerometer X'], datac[0]['Accelerometer X'], alternative="greater"))
print(ttest_ind(datax[0]['Accelerometer X'], datac[0]['Accelerometer X'], alternative="two-sided"))
print(ttest_ind(datax[0]['Accelerometer Y'], datac[0]['Accelerometer Y'], alternative="less"))
print(ttest_ind(datax[0]['Accelerometer Y'], datac[0]['Accelerometer Y'], alternative="greater"))
print(ttest_ind(datax[0]['Accelerometer Y'], datac[0]['Accelerometer Y'], alternative="two-sided"))
print(ttest_ind(datax[0]['Accelerometer Z'], datac[0]['Accelerometer Z'], alternative="less"))
print(ttest_ind(datax[0]['Accelerometer Z'], datac[0]['Accelerometer Z'], alternative="greater"))
print(ttest_ind(datax[0]['Accelerometer Z'], datac[0]['Accelerometer Z'], alternative="two-sided"))
print(ttest_ind(datax[0]['Gyroscope X'], datac[0]['Gyroscope X'], alternative="less"))
print(ttest_ind(datax[0]['Gyroscope X'], datac[0]['Gyroscope X'], alternative="greater"))
print(ttest_ind(datax[0]['Gyroscope X'], datac[0]['Gyroscope X'], alternative="two-sided"))
print(ttest_ind(datax[0]['Gyroscope Y'], datac[0]['Gyroscope Y'], alternative="less"))
print(ttest_ind(datax[0]['Gyroscope Y'], datac[0]['Gyroscope Y'], alternative="greater"))
print(ttest_ind(datax[0]['Gyroscope Y'], datac[0]['Gyroscope Y'], alternative="two-sided"))
print(ttest_ind(datax[0]['Gyroscope Z'], datac[0]['Gyroscope Z'], alternative="less"))
print(ttest_ind(datax[0]['Gyroscope Z'], datac[0]['Gyroscope Z'], alternative="greater"))
print(ttest_ind(datax[0]['Gyroscope Z'], datac[0]['Gyroscope Z'], alternative="two-sided"))
print(ttest_ind(datax[0]['Magnetometer X'], datac[0]['Magnetometer X'], alternative="less"))
print(ttest_ind(datax[0]['Magnetometer X'], datac[0]['Magnetometer X'], alternative="greater"))
print(ttest_ind(datax[0]['Magnetometer X'], datac[0]['Magnetometer X'], alternative="two-sided"))
print(ttest_ind(datax[0]['Magnetometer Y'], datac[0]['Magnetometer Y'], alternative="less"))
print(ttest_ind(datax[0]['Magnetometer Y'], datac[0]['Magnetometer Y'], alternative="greater"))
print(ttest_ind(datax[0]['Magnetometer Y'], datac[0]['Magnetometer Y'], alternative="two-sided"))
print(ttest_ind(datax[0]['Magnetometer Z'], datac[0]['Magnetometer Z'], alternative="less"))
print(ttest_ind(datax[0]['Magnetometer Z'], datac[0]['Magnetometer Z'], alternative="greater"))
print(ttest_ind(datax[0]['Magnetometer Z'], datac[0]['Magnetometer Z'], alternative="two-sided"))
a = features[0:20000]
b = features[180000:200000]
tstats = []
pvalues = []
tstats2 = []
pvalues2 = []
#T-test compare Feature sets 
for i in range(1,192):
  t_statistic, pvalue = ttest_ind(a.iloc[:,i],b.iloc[:,i], alternative="less")
  t_statistic2, pvalue2 = ttest_ind(a.iloc[:,i],b.iloc[:,i], alternative="greater")
  tstats.append(t_statistic)
  pvalues.append(pvalue)
  tstats2.append(t_statistic2)
  pvalues2.append(pvalue2)
  print(feature_names[i], t_statistic, pvalue)
  print(feature_names[i], t_statistic2, pvalue2)




497
Ttest_indResult(statistic=9.43741357985779e-05, pvalue=0.5000376495191432)
Ttest_indResult(statistic=9.43741357985779e-05, pvalue=0.4999623504808568)
Ttest_indResult(statistic=9.43741357985779e-05, pvalue=0.9999247009617136)
Ttest_indResult(statistic=-35.32244980166652, pvalue=4.1157298802613117e-268)
Ttest_indResult(statistic=-35.32244980166652, pvalue=1.0)
Ttest_indResult(statistic=-35.32244980166652, pvalue=8.231459760522623e-268)
Ttest_indResult(statistic=-42.797739164229974, pvalue=0.0)
Ttest_indResult(statistic=-42.797739164229974, pvalue=1.0)
Ttest_indResult(statistic=-42.797739164229974, pvalue=0.0)
Ttest_indResult(statistic=-2.4499923858813366, pvalue=0.007145798115236358)
Ttest_indResult(statistic=-2.4499923858813366, pvalue=0.9928542018847637)
Ttest_indResult(statistic=-2.4499923858813366, pvalue=0.014291596230472716)
Ttest_indResult(statistic=0.18539574012656582, pvalue=0.5735400486325506)
Ttest_indResult(statistic=0.18539574012656582, pvalue=0.4264599513674493)
Ttest_i

In [62]:
df = pd.DataFrame(features)
#Shuffle the dataset 
df['labels'] = labels
df['sex_labels'] = sex_labels.values.tolist()
x = df.sample(frac=1)
y = x['labels']
print(x)
y2 = x['sex_labels']
x = x.drop('labels', axis=1)
#x = x.drop('0', axis=1)
x = x.drop('sex_labels', axis=1)
print(x)

               1         2         3         4         5         6         7  \
87548  -0.004182  0.003891  0.001801  0.001801  0.187772 -0.636905 -0.716797   
218315 -0.028068  0.026090  0.010655  0.010639 -0.371153  0.148989 -0.550293   
446969 -2.382537  1.654779  0.442743  0.442741 -0.649694  9.788045 -5.442871   
354054 -0.223540  0.302112  0.098003  0.097301  0.372724  0.601304 -1.136230   
350852 -0.185375  0.077864  0.053587  0.052831 -1.171339  1.537632 -0.610840   
...          ...       ...       ...       ...       ...       ...       ...   
81963  -0.300914  0.761571  0.162845  0.162700  1.532393  4.471622 -1.276855   
363127 -0.004618  0.005018  0.001963  0.001963  0.049529 -0.412287 -0.574219   
397282 -0.005844  0.005860  0.002021  0.002020  0.096489  0.245493 -0.573242   
196157 -1.700516  4.367524  0.935456  0.931640  1.693654  5.092993 -4.338867   
379444 -0.005470  0.005527  0.002073  0.002072  0.402820  0.527741 -0.573730   

               8         9        10  .

In [ ]:
#Feature Selection: K-best, tune K as hyperparameter
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn import metrics


featurenums = [3, 6, 12, 24, 36, 48, 60]
estimators = [50, 100, 200, 300, 400]
featuresets = []

for i in featurenums:
  featuresets.append(featureSelection(x, i, y, 2))
test_acc = []
val_acc = []
for n in range(len(estimators)):
  for i in range(len(featuresets)):
    X_train, X_test, y_train, y_test = train_test_split(featuresets[i], y, test_size=0.2)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
    clf=RandomForestClassifier(n_estimators=estimators[n])
    #Train the model using the training sets
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    yval_pred=clf.predict(X_val)
    print("Test Acc of {} & {} : {} Validation Acc {}".format(featurenums[i], estimators[n], metrics.accuracy_score(y_test, y_pred),  metrics.accuracy_score(y_val, yval_pred)))

%x : %x 3 50 1.0
%x : %x 3 50 1.0
%x : %x 6 50 1.0
%x : %x 6 50 1.0
%x : %x 12 50 1.0
%x : %x 12 50 1.0
%x : %x 24 50 1.0
%x : %x 24 50 1.0
%x : %x 36 50 1.0
%x : %x 36 50 1.0
%x : %x 48 50 1.0
%x : %x 48 50 1.0
%x : %x 60 50 1.0
%x : %x 60 50 1.0
%x : %x 3 100 1.0
%x : %x 3 100 1.0
%x : %x 6 100 1.0
%x : %x 6 100 1.0
%x : %x 12 100 1.0
%x : %x 12 100 1.0
%x : %x 24 100 1.0
%x : %x 24 100 1.0
%x : %x 36 100 1.0
%x : %x 36 100 1.0
%x : %x 48 100 1.0
%x : %x 48 100 1.0
%x : %x 60 100 1.0
%x : %x 60 100 1.0
%x : %x 3 200 1.0
%x : %x 3 200 1.0
%x : %x 6 200 1.0
%x : %x 6 200 1.0
%x : %x 12 200 1.0
%x : %x 12 200 1.0
%x : %x 24 200 1.0
%x : %x 24 200 1.0
%x : %x 36 200 1.0
%x : %x 36 200 1.0
%x : %x 48 200 1.0
%x : %x 48 200 1.0
%x : %x 60 200 1.0
%x : %x 60 200 1.0
%x : %x 3 300 1.0
%x : %x 3 300 1.0
%x : %x 6 300 1.0
%x : %x 6 300 1.0
%x : %x 12 300 1.0
%x : %x 12 300 1.0
%x : %x 24 300 1.0
%x : %x 24 300 1.0
%x : %x 36 300 1.0
%x : %x 36 300 1.0
%x : %x 48 300 1.0
%x : %x 48 300 1.0
%x :

In [ ]:
#Sex Identification task
estimators = [50, 100, 200, 300, 400]
featuresets = []
for i in featurenums:
  featuresets.append(featureSelection(x, i, y2, 2))
test_acc = []
val_acc = []
for n in range(len(estimators)):
  for i in range(len(featuresets)):
    X_train, X_test, y_train, y_test = train_test_split(featuresets[i], y2, test_size=0.2)
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)
    clf=RandomForestClassifier(n_estimators=estimators[n])
    #Train the model using the training sets
    clf.fit(X_train,y_train)
    y_pred=clf.predict(X_test)
    yval_pred=clf.predict(X_val)
    print("Test Acc of {} & {} : {}".format(featurenums[i], estimators[n], metrics.accuracy_score(y_test, y_pred)))
    print("Validation Acc of {} & {} : {}".format(featurenums[i], estimators[n], metrics.accuracy_score(y_val, yval_pred)))

Test Acc of 3 & 50 : 0.908858937450929
Validation Acc of 3 & 50 : 0.9090988397452674
Test Acc of 6 & 50 : 0.9101347814708192
Validation Acc of 6 & 50 : 0.9082591817150834
Test Acc of 12 & 50 : 0.9090552211462968
Validation Acc of 12 & 50 : 0.9090116025473262
Test Acc of 24 & 50 : 0.9091315536944954
Validation Acc of 24 & 50 : 0.9098730698769956
Test Acc of 36 & 50 : 0.9063508680101195
Validation Acc of 36 & 50 : 0.9101238768210765
Test Acc of 48 & 50 : 0.9088153188519584
Validation Acc of 48 & 50 : 0.9086408444560761
Test Acc of 60 & 50 : 0.9075176655325831
Validation Acc of 60 & 50 : 0.9084990840094216
Test Acc of 3 & 100 : 0.9084990840094216
Validation Acc of 3 & 100 : 0.9095568350344587
Test Acc of 6 & 100 : 0.9092296955421791
Validation Acc of 6 & 100 : 0.9084336561109657
Test Acc of 12 & 100 : 0.9096767861816278
Validation Acc of 12 & 100 : 0.9090225071970688
Test Acc of 24 & 100 : 0.9077793771264067
Validation Acc of 24 & 100 : 0.9091751722934659
Test Acc of 36 & 100 : 0.91005844

In [ ]:
#Train Random Forest Classifier but no feature selection at all
# hyperparameter tuning: trees  100, 200, 300, etc.
for n in range(len(estimators)):
  X_train, X_test, y_train, y_test = train_test_split(x, y2, test_size=0.2)
  X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25)

  clf=RandomForestClassifier(n_estimators=estimators[n])

  #Train the model using the training sets
  clf.fit(X_train,y_train)

  y_pred=clf.predict(X_test)
  yval_pred=clf.predict(X_val)

  # Model Accuracy
  print("Accuracy with {} estimators:".format(n),metrics.accuracy_score(y_test, y_pred), metrics.accuracy_score(y_val, yval_pred))

Accuracy with 0 estimators: 1.0 1.0
Accuracy with 1 estimators: 1.0 1.0
Accuracy with 2 estimators: 1.0 1.0
Accuracy with 3 estimators: 1.0 1.0
Accuracy with 4 estimators: 1.0 1.0


In [66]:
from sklearn.metrics import classification_report
from sklearn import tree
import graphviz
#Train Random Forest Classifier
def fit_CART(X, y, criterion, splitter, mdepth, clweight, minleaf):

    # Create training and testing samples
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # Fit the model
    model = tree.DecisionTreeClassifier(criterion=criterion, 
                                        splitter=splitter, 
                                        max_depth=mdepth,
                                        class_weight=clweight,
                                        min_samples_leaf=minleaf, 
                                        random_state=0, 
                                  )
    clf = model.fit(X_train, y_train)

    # Predict class labels on training data
    pred_labels_tr = model.predict(X_train)
    # Predict class labels on a test data
    pred_labels_te = model.predict(X_test)

    # Tree summary and model evaluation metrics
    print('*************** Tree Summary ***************')
    print('Classes: ', clf.classes_)
    print('Tree Depth: ', clf.tree_.max_depth)
    print('No. of leaves: ', clf.tree_.n_leaves)
    print('No. of features: ', clf.n_features_in_)
    print('--------------------------------------------------------')
    print("")
    
    print('*************** Evaluation on Test Data ***************')
    score_te = model.score(X_test, y_test)
    print('Accuracy Score: ', score_te)
    # Look at classification report to evaluate the model
    print(classification_report(y_test, pred_labels_te))
    print('--------------------------------------------------------')
    print("")
    
    print('*************** Evaluation on Training Data ***************')
    score_tr = model.score(X_train, y_train)
    print('Accuracy Score: ', score_tr)
    #print(classification_report(y_train, pred_labels_tr))
    print('--------------------------------------------------------')
    

fit_CART(x, y, 'gini', 'best', mdepth=3, clweight=None, minleaf=1000)

*************** Tree Summary ***************
Classes:  ['Control' 'LLA']
Tree Depth:  2
No. of leaves:  3
No. of features:  192
--------------------------------------------------------

*************** Evaluation on Test Data ***************
Accuracy Score:  0.9987677745790805
              precision    recall  f1-score   support

     Control       1.00      1.00      1.00     55769
         LLA       1.00      1.00      1.00     35935

    accuracy                           1.00     91704
   macro avg       1.00      1.00      1.00     91704
weighted avg       1.00      1.00      1.00     91704

--------------------------------------------------------

*************** Evaluation on Training Data ***************
Accuracy Score:  0.9989068058830746
--------------------------------------------------------
